In [1]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import csv
import requests
import json
import datetime
%matplotlib inline

day = datetime.datetime.today().day
month = datetime.datetime.today().month
year = datetime.datetime.today().year

api_key = 's2dumBY1WIgOCUNbPhFOyKBiUdSAdR4d'
alpha_key = 'FPL5DH1JLA759DX4'

def get_data(symbol):
    r = requests.get(f'https://api.iextrading.com/1.0/stock/{symbol}/chart/1y')
    pr = json.loads(r.content)
    df = pd.DataFrame(pr)
    df.set_index('date', inplace = True)
    return df


buy_stocks = []
sell_stocks = []
watchlist_d = {}
symbols = []


csv_read = csv.reader(open('companylist_all.csv'))
for row in csv_read:
    if row[0] != 'Symbol':
        symbols.append(row[0])
        
master_df_d = {} 
print(len(symbols))
for symbol in symbols: 
    try: 
        df = get_data(symbol)
        df['close'] = df['close'].astype(float)
        df['volume'] = df['volume'].astype(float)
        df['prev_price'] = df['close'] + df['change']
        df['gain'] = df['change']
        df['loss'] = df['change']
        df['gain'].where(df['gain']>0, other = 0.0, inplace = True)
        df['loss'].where(df['loss']<0 , other = 0.0, inplace = True)
        df['loss'] = abs(df['loss'])
        df['avg_volume'] = df['volume'].rolling(50).mean()
        df['avg_gain'] = df['gain'].rolling(14).mean()
        df['avg_loss'] = df['loss'].rolling(14).mean()
        df['prev_avg_gain'] = df['avg_gain']
        df['prev_avg_loss'] = df['avg_loss']
        if df.iloc[-1]['avg_volume'] > 3000000:
            df.iloc[15]['avg_gain'] = ((df.iloc[15]['prev_avg_gain']*13)+df.iloc[15]['gain'])/14
            df.iloc[15]['avg_loss'] = ((df.iloc[15]['prev_avg_loss']*13)+df.iloc[15]['loss'])/14
            for row in df.index[16:]:
                try:
                    df.loc[row]['avg_gain'] = ((df.loc[lastrow]['avg_gain']*13)+df.loc[row]['gain'])/14 
                    df.loc[row]['avg_loss'] = ((df.loc[lastrow]['avg_loss']*13)+df.loc[row]['loss'])/14
                    lastrow = row
                except:
                    next
            df['rs'] = df['avg_gain']/df['avg_loss']
            df['rsi'] = 100 - (100/(1.0+df['rs']))
            df['prev_close'] = df['close'].shift(1)
            df['fast'] = df['close'].ewm(span=12).mean()
            df['slow'] = df['close'].ewm(span=26).mean()
            df['macd'] = df['fast']-df['slow']
            df['signal'] = df['macd'].ewm(span=9).mean()
            df['200MA'] = df['close'].rolling(200).mean()
            df['50MA'] = df['close'].rolling(50).mean()

            watchlist_d[symbol] = {'rsi': df.iloc[-1]['rsi'] ,
                                   'prev_close' : df.iloc[-1]['prev_close'],
                                   'close' : df.iloc[-1]['close'],
                                   'avg_volume' : df.iloc[-1]['avg_volume'],
                                   'signal' : df.iloc[-1]['signal'],
                                   'macd' : df.iloc[-1]['macd'],
                                   'last_rsi' : df.iloc[-2]['rsi'],
                                   'last_macd' : df.iloc[-2]['macd'] - df.iloc[-2]['signal'],
                                   '200ma' : df.iloc[-1]['200MA'],
                                   '50ma' : df.iloc[-1]['50MA'],
                                   'trend' : df.iloc[-1]['50MA'] - df.iloc[-2]['50MA'],
                                    'volume': df.iloc[-1]['volume']}
        else:
            next
        
    except:
        print(f'cannot get data for {symbol}')
        next


print(f'Getting ratios for {str(len(watchlist_d))} stocks.')

for symbol in watchlist_d:
    try:
        find_id = f'https://simfin.com/api/v1/info/find-id/ticker/{symbol}?api-key={api_key}'
        r = requests.get(find_id)
        pr = json.loads(r.content)
        sim_id = pr[0]['simId']
        r = requests.get(f'https://simfin.com/api/v1/companies/id/{str(sim_id)}/ratios?api-key={api_key}')
        pr = json.loads(r.content)
        for item in pr:
            if item['indicatorName'] == 'Price to Earnings Ratio':
                watchlist_d[symbol]['pe'] = item['value']
            elif item['indicatorName'] == 'Price to Book Value':
                watchlist_d[symbol]['pb'] = item["value"]
            elif item['indicatorName'] == 'Free Cash Flow':
                watchlist_d[symbol]['free_cash_flow'] = item['value']
            elif item['indicatorName'] == 'Enterprise Value':
                watchlist_d[symbol]['enterprise_value'] = item['value']
        try:
            watchlist_d[symbol]['free_cash_flow_yield'] = float(watchlist_d[symbol]['free_cash_flow'])/float(watchlist_d[symbol]['enterprise_value'])
        except:
            next
    except:
        print(f'cannot find ratios for {symbol}')


for symbol in watchlist_d:
    try:
        if watchlist_d[symbol]['rsi'] <= 30 and 0<float(watchlist_d[symbol]['pe']) < 20:
            buy_stocks.append(symbol)
        elif watchlist_d[symbol]['rsi'] >= 75 and float(watchlist_d[symbol]['pb']) > 5:
            sell_stocks.append(symbol)
    except:
        pass
print('buy stocks')
for symbol in buy_stocks:
    
    print(symbol)
    print(watchlist_d[symbol])
print('\n\n')
print('sell stocks')
for symbol in sell_stocks:
    print(symbol)
    print(watchlist_d[symbol])
    

    

6895


c:\users\rob\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\rob\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


cannot get data for MITT^A
cannot get data for MITT^B
cannot get data for ALP^Q
cannot get data for ARE^D
cannot get data for NCV^A
cannot get data for NCZ^A
cannot get data for ALL^A
cannot get data for ALL^B
cannot get data for ALL^D
cannot get data for ALL^E
cannot get data for ALL^F
cannot get data for ALL^G
cannot get data for ALLY^A
cannot get data for AMH^D
cannot get data for AMH^E
cannot get data for AMH^F
cannot get data for AMH^G
cannot get data for AMH^H
cannot get data for AIG.WS
cannot get data for AFSI^A
cannot get data for AFSI^B
cannot get data for AFSI^C
cannot get data for AFSI^D
cannot get data for AFSI^E
cannot get data for AFSI^F
cannot get data for NLY^C
cannot get data for NLY^D
cannot get data for NLY^F
cannot get data for NLY^G
cannot get data for NLY^H
cannot get data for ANH^A
cannot get data for ANH^B
cannot get data for ANH^C
cannot get data for AIV^A
cannot get data for ARI^C
cannot get data for APO^A
cannot get data for APO^B
cannot get data for ABR^A
ca

c:\users\rob\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\rob\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


cannot get data for NNN^E
cannot get data for NNN^F
cannot get data for NSA^A
cannot get data for NM^G
cannot get data for NM^H
cannot get data for NAV^D
cannot get data for NFC.U
cannot get data for NFC.WS
cannot get data for NYCB^A
cannot get data for NYCB^U
cannot get data for NEE^I
cannot get data for NEE^J
cannot get data for NEE^K
cannot get data for NEE^R
cannot get data for NGL^B
cannot get data for NMK^B
cannot get data for NMK^C
cannot get data for NS^A
cannot get data for NS^B
cannot get data for NS^C
cannot get data for OAK^A
cannot get data for OAK^B
cannot get data for OFG^A
cannot get data for OFG^B
cannot get data for OFG^D
cannot get data for OIBR~$
cannot get data for OMAD.U
cannot get data for OMAD.WS
cannot get data for PRE^F
cannot get data for PRE^G
cannot get data for PRE^H
cannot get data for PRE^I
cannot get data for PEB^C
cannot get data for PEB^D
cannot get data for PEI^B
cannot get data for PEI^C
cannot get data for PEI^D
cannot get data for PMT^A
cannot get

cannot find ratios for FHN
cannot find ratios for AG
cannot find ratios for GGB
cannot find ratios for GSK
cannot find ratios for GFI
cannot find ratios for GG
cannot find ratios for GPK
cannot find ratios for HMY
cannot find ratios for HL
cannot find ratios for HUYA
cannot find ratios for IAG
cannot find ratios for IBN
cannot find ratios for INFY
cannot find ratios for ING
cannot find ratios for ITUB
cannot find ratios for KGC
cannot find ratios for KOS
cannot find ratios for LPI
cannot find ratios for LVS
cannot find ratios for LYG
cannot find ratios for MPW
cannot find ratios for MBT
cannot find ratios for NBR
cannot find ratios for SID
cannot find ratios for NYCB
cannot find ratios for NIO
cannot find ratios for NOK
cannot find ratios for OAS
cannot find ratios for PBR
cannot find ratios for PBR.A
cannot find ratios for SBGL
cannot find ratios for SU
cannot find ratios for SPN
cannot find ratios for TAHO
cannot find ratios for TSM
cannot find ratios for TAL
cannot find ratios for T